# ⬇️ Imports

In [ ]:
##adding something

In [ ]:
%load_ext autoreload

%autoreload 2

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import seaborn as sns

import sys
sys.path.append('/home/anouchka/code/qadnguyen/realdata')
from prop_value.ml_logic.preprocessor import preprocess_data, clean_data


import warnings
warnings.filterwarnings('ignore')


# ✅ Downloading the data

In [ ]:
# # Get the data from the csv file
# dvf_full_raw = pd.read_csv('../raw_data/dvf_full_raw.csv')

# dvf_full_raw.shape

In [ ]:
df_93_raw = pd.read_csv('../raw_data/dvf_93.csv')

In [ ]:
# df = pd.read_csv('../raw_data/dvf_full_raw.csv')

# 🔎 Exploring 1 region

In [ ]:
df_93_raw.columns

# 🔅 Preprocessing

## 🧹 Keep only useful columns and rows and translate

### Removing columns

In [ ]:
keep_col =['id_mutation',
            'date_mutation',
            'nature_mutation',
            'valeur_fonciere',
            'code_postal',
            'code_commune',
            'code_departement',
            'type_local',
            'surface_reelle_bati',
            'nombre_pieces_principales',
            'surface_terrain',
            'longitude',
            'latitude']
df_93 = df_93_raw[keep_col]

df_93.info()

### Translate column-names

In [ ]:
df_93.columns = ['id', 'date', 'built', 'price', 'postal_code',
                'city', 'region', 'property_type',
                'living_area', 'number_of_rooms', 'land_area',
                'longitude', 'latitude']

### Remove non-representative rows

In [ ]:
df_93.shape

We want to keep only :
- Houses and appartments + Dépendances
- Normal sales and off-plan sales
- Sales with only one unit XXX

In [ ]:
df_93.property_type.unique()

In [ ]:
print('Shape before the masks : ', df_93.shape)
print('Shape only normal sales', df_93[((df_93['built'] == "Vente") | (df_93['built'] == "Vente en l'état futur d'achèvement"))].shape)
print('Shape only house and appartments + dependencies', df_93[((df_93['property_type'] == 'Appartement') | (df_93['property_type'] == 'Maison')| (df_93['property_type'] == 'Dépendance'))].shape)

In [ ]:
df_useful = df_93[((df_93['built'] == "Vente") | (df_93['built'] == "Vente en l'état futur d'achèvement")) &
                    ((df_93['property_type'] == 'Appartement') | (df_93['property_type'] == 'Maison')  | (df_93['property_type'] == 'Maison')| (df_93['property_type'] == 'Dépendance'))]
df_useful.shape

In [ ]:
## checking that the masks worked correctly
print(df_useful['built'].unique())
print(df_useful['property_type'].unique())


### Translating the values

In [ ]:
trans_dict_built = {'Vente' : 'built',
                    "Vente en l'état futur d'achèvement" : 'off-plan'}
trans_dict_type = {'Appartement' : 'appartment',
                   'Maison' : 'house',
                   'Dépendance' : 'dep'}
df_useful= df_useful.replace({'built' : trans_dict_built,
                             'property_type' : trans_dict_type})

In [ ]:
## checking that the translation worked correctly
print(df_useful['built'].unique())
print(df_useful['property_type'].unique())

In [ ]:
print(f'Number of elements that have been sold : {df_useful.shape[0]}')

In [ ]:
print(f'Number of unique sale ids : {df_useful.id.nunique()}.')

# 🤓 Feature engineering

1. number of dependencies
2. garden or not --> ???

## 🛖 number of dependencies

In [ ]:
mask_house = df_useful.property_type == 'house'
mask_app = df_useful.property_type == 'appartment'
mask_dep = df_useful.property_type == 'dep'

In [ ]:
#find the number of houses per id
num_of_houses_df = df_useful[['id','property_type']][mask_house].groupby(by = 'id', as_index = False).count()
num_of_houses_df.value_counts()
num_of_houses_df.columns = ['id', 'nb_of_house']
num_of_houses_df.head()

In [ ]:
#find the number of appartments per id
num_of_app_df = df_useful[['id','property_type']][mask_app].groupby(by = 'id', as_index = False).count()
num_of_app_df.value_counts()
num_of_app_df.columns = ['id', 'nb_of_app']
num_of_app_df.head()

In [ ]:
#find the number of dep per id
num_of_dep_df = df_useful[['id','property_type']][mask_dep].groupby(by = 'id', as_index = False).count()
num_of_dep_df.value_counts()
num_of_dep_df.columns = ['id', 'nb_of_dep']
num_of_dep_df.head()

In [ ]:
num_of_dep_df.id.nunique()

➕ adding columns to df_useful

In [ ]:
df_feat = df_useful.merge(num_of_houses_df, on = 'id', how = 'outer')
df_feat = df_feat.merge(num_of_app_df, on = 'id', how = 'outer')
df_feat = df_feat.merge(num_of_dep_df, on = 'id', how = 'outer')
df_feat.sample(5)

In [ ]:
df_feat.fillna({'nb_of_house' : 0, 'nb_of_app' : 0, 'nb_of_dep' : 0, 'land_area' : 0}, inplace=True)

In [ ]:
# remove lines with multiple houses or appartments in same sale_id
df_feat['nb_of_units'] = df_feat.nb_of_app+df_feat.nb_of_house
mask_units = (df_feat['nb_of_units'] == 1)
df_filtered = df_feat[mask_units]
df_filtered.shape

In [ ]:
df_filtered.sample(10)

In [ ]:
#filter out the dependencies
filter_type = (df_filtered['property_type'] != 'dep')
df_filtered = df_filtered[filter_type]
df_filtered.shape

In [ ]:
df_filtered_drop = df_filtered.drop(columns = ['nb_of_units', 'nb_of_app', 'nb_of_house'])
df_filtered_drop.shape
df_filtered_drop.sample(5)

In [ ]:
df_filtered_drop.duplicated().sum()

## 🌼 Adding garden feature ? TODO : Nan = 0, other number = 1

❓❔❓ relation between land_area and living_area

In [ ]:
df_filtered_drop.duplicated().sum()

In [ ]:
df_filtered_drop.isnull().sum()/len(df_filtered_drop) # 76 % of land_area is null

In [ ]:
mask_house = df_filtered_drop.property_type == 'house'
df_filtered_drop[mask_house].isnull().sum()/len(df_filtered_drop[mask_house]) # 13 % of land_area is null

In [ ]:
mask_appartments = df_filtered_drop.property_type == 'appartment'
df_filtered_drop[mask_appartments].isnull().sum()/len(df_filtered_drop[mask_appartments]) # 99 % of land_area is null

In [ ]:
########################################### TO DO : garden 1 or 0

## 🚮 Dropping duplicates and NaNs

➡️ 76% of the land_area is Null... --> drop the column land_area ?

In [ ]:
df_final = df_filtered_drop.drop(columns = ['land_area', 'id'])
df_clean = df_final.dropna()
df_clean.shape

## 🧮 Count summary

In [ ]:
print('Number of lines before the masks : ', df_93.shape[0])
print('Number of lines of only normal sales : ', df_93[((df_93['built'] == "Vente") | (df_93['built'] == "Vente en l'état futur d'achèvement"))].shape[0])
print('Number of lines of only house and appartments + dependencies : ', df_93[((df_93['property_type'] == 'Appartement') | (df_93['property_type'] == 'Maison')| (df_93['property_type'] == 'Dépendance'))].shape[0])
print(f'Number of unique sale ids : {df_clean.shape[0]}')

## 💰 Create € / sq m

In [ ]:
df_clean[['price', 'living_area']] = df_clean[['price', 'living_area']].apply(lambda x: pd.to_numeric(x, errors='coerce').astype('float64'))

df_clean['price_per_m2'] = df_clean.price / df_clean.living_area

In [ ]:
df_clean.sample(5)

## 🦖 Changing everything to the right type

In [ ]:
df_clean.columns

In [ ]:
#changing evething to the right type --> TO DO:should this go to params?
col_float = ['price', 'longitude', 'latitude', 'living_area', 'price_per_m2' ]
col_string = ['built','city', 'region','property_type']
col_date = ['date']
col_int = ['postal_code', 'nb_of_dep', 'number_of_rooms']
#formating data types
df_clean[col_float] = df_clean[col_float].apply(lambda x: pd.to_numeric(x, errors='coerce').astype('float64'))
df_clean[col_date]= df_clean[col_date].apply(lambda x: pd.to_datetime(x, errors='coerce'))
df_clean[col_int]= df_clean[col_int].apply(lambda x: pd.to_numeric(x, errors='coerce').astype('int64'))

df_clean.info()

## drop outliers
1. find price per sq m - DONE
2. group by each postal code
3. percentile 96 % of this new feature
4. drop everything above it

In [ ]:
df_quantile = df_clean[['postal_code', 'price_per_m2']].groupby(by = df_clean.postal_code, as_index=False).quantile(.95)
df_quantile.columns = ['postal_code', 'p95']
df_quantile.head()

In [ ]:
df_clean_merged = df_quantile.merge(right = df_clean, on = 'postal_code')
df_clean_merged.head()

In [ ]:
df_clean_merged.shape

In [ ]:
# drop the outliers
mask_outliers = (df_clean_merged.price_per_m2 < df_clean_merged.p95)
df_without_outliers = df_clean_merged[mask_outliers]

In [ ]:
df_clean_merged.shape, df_without_outliers.shape

In [ ]:
# def filter_top_4_percent(df, column_name):
#     threshold = df[column_name].quantile(0.96)
#     return df[df[column_name] <= threshold]

# df_without_outliers_2 = filter_top_4_percent(df_clean_merged, '')

In [ ]:
df_without_outliers.shape[0]/df_clean_merged.shape[0]

In [ ]:
df_without_outliers.head()

In [ ]:
df_clean_merged.head()

In [ ]:
df_clean_merged.price_per_m2.sort_values(ascending = False).head(20)

In [ ]:
df_without_outliers.price_per_m2.sort_values(ascending = False).head(30)

In [ ]:
# drop p95
df_without_outliers = df_without_outliers.drop(columns=['p95', 'price_per_m2'])

In [ ]:
df_without_outliers.info()

Dealing date_column

In [ ]:
df_without_outliers.date.head()

In [ ]:
df_without_outliers.date.dt.day.head()

In [ ]:
df_without_outliers['day'] = df_without_outliers.date.dt.day

df_without_outliers['day_sin'] = np.sin(2 * np.pi * df_without_outliers['day']/31.0)
df_without_outliers['day_cos'] = np.cos(2 * np.pi * df_without_outliers['day']/31.0)

In [ ]:
df_without_outliers['month'] = df_without_outliers.date.dt.month

df_without_outliers['month_sin'] = np.sin(2 * np.pi * df_without_outliers['month']/24.0)
df_without_outliers['month_cos'] = np.cos(2 * np.pi * df_without_outliers['month']/24.0)

In [ ]:
df_without_outliers['year'] = df_without_outliers.date.dt.year


In [ ]:
df_without_outliers = df_without_outliers.drop(columns = ['day', 'month'])

In [ ]:
df_without_outliers.year

## ✨ Using the functions

In [ ]:
# check if function is working :
import sys
sys.path.append('/home/anouchka/code/qadnguyen/realdata')
from prop_value.ml_logic.preprocessor import clean_data

clean_data(df_93_raw, percentile = 0.95).shape, df_without_outliers.shape

In [ ]:
df_without_outliers.info()

In [ ]:
# check if function is working :
import sys
sys.path.append('/home/anouchka/code/qadnguyen/realdata')
from prop_value.ml_logic.preprocessor import preprocess_data

data = clean_data(df_93_raw, percentile = 0.95)
print(f'shape of the cleaned data : {data.shape}')
print(data.info())

X_train_preproc_TEST, X_test_preproc_TEST, y_train_TEST, y_test_TEST, X_all_TEST, y_all_TEST, df_full_TEST = preprocess_data(data)
X_all_TEST.info()


# 🐧 Modelling

## 🔎 Exploring some more

In [ ]:
df_clean.columns

In [ ]:
# df_cleaned.shape # (1029674, 12)

In [ ]:
# df_cleaned[mask_price].shape # (1027158, 12)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize = (12,5))
sns.scatterplot(df_clean['living_area'], df_clean['price'],hue = df_clean['property_type'],alpha = 0.5)
plt.show()

In [ ]:
plt.figure(figsize = (12,4))
sns.boxplot(df_clean.price)

In [ ]:
df_clean.head()

In [ ]:
# fig, axs = plt.subplots(1, 2, figsize=(12, 5))  # 1 row, 2 columns
# mask_price = df_clean['price']<2000000 #smaller than 2 milion
# mask_area = df_clean['living_area']<500 # smaller than 500 sq meters
# df_without_outliers = df_clean[mask_price & mask_area]

# Plot Living area vs Sale price
plt.figure(figsize = (12,5))
sns.scatterplot(df_without_outliers['living_area'], df_without_outliers['price'],hue = df_without_outliers['property_type'],alpha = 0.5)
plt.show()

In [ ]:
df_without_outliers['property_type'].value_counts()

In [ ]:
sns.heatmap(df_without_outliers.corr(), cmap = 'vlag', annot = True)

In [ ]:
df_without_outliers.info()

## Take a look at the final data

## 🐟 Naive baseline model

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
mask_price = df_clean['price'] <2000000
df_without_outliers = df_clean[mask_price]

In [ ]:
X = df_without_outliers.drop(columns=['price'])
y = df_without_outliers['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
# plt.figure(figsize=(12,4))
# sns.boxplot(y)


In [ ]:
mean_price = y_train.mean()
print(f"The mean price of {round(mean_price)} €")

median_price = y_train.median()
print(f"The median price of {round(median_price)} €")

In [ ]:
mean_error = y_test - mean_price
rmse_mean = ((mean_error**2).mean())**(1/2)
print(f'The rmse when always predicting the mean : {round(rmse_mean)} €.')

In [ ]:
median_error = y_test - median_price
rmse_median = ((median_error**2).mean())**(1/2)
print(f'The rmse when always predicting the median : {round(rmse_median)} €.')

## 🐸 Model

In [ ]:
df_full_clean = pd.read_pickle('../raw_data/dvf_full_cleaned.pkl')


In [ ]:
df_full_clean.price.mean()

## 💢 encode the data

In [ ]:
# df_93_clean = clean_data(df_93_raw)

In [ ]:
X_train_preproc, X_test_preproc, y_train, y_test, X_all, y_all, df_full = preprocess_data(df_full_clean)

In [ ]:
df_full.shape

In [ ]:
import sys
sys.path.append('/home/anouchka/code/qadnguyen/realdata')
from prop_value.ml_logic.preprocessor import preprocess_data
X_train_preproc, X_test_preproc, y_train, y_test, X_all, y_all, df_full = preprocess_data(df_full_clean)

In [ ]:
# model-specific imports
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import validation_curve
from sklearn.model_selection import cross_val_score


## 🌳 RandomForest on data without outliers (df_without_ouliers)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
model_rfr = RandomForestRegressor()

model_rfr.fit(X_train_preproc, y_train)
y_pred = model_rfr.predict(X_test_preproc)
mse = mean_squared_error(y_test, y_pred)
rmse = mse**(1/2)
print(f'Root Mean Squared Error: {rmse}')
print( 'Train score (R²):', model_rfr.score(X_test_preproc, y_test))
print('Cross val score mean (R²)', cross_val_score(model_rfr, X_all, y_all, cv = 5).mean())

## 🪃 XGBoost on data without outliers

In [ ]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
model_xgb = XGBRegressor()

model_xgb.fit(X_train_preproc, y_train)
y_pred = model_xgb.predict(X_test_preproc)
mse_b = mean_squared_error(y_pred, y_test)
rmse_b = mse_b**(1/2)
print(f'Root Mean Squared Error: {rmse_b} €')
print( 'Train score (R²) : ', model_xgb.score(X_train_preproc, y_train))
print( 'Test score (R²) : ', model_xgb.score(X_test_preproc, y_test))

In [ ]:
(((y_test.price-y_pred)**2).mean())**(1/2)

In [ ]:
df_y_pred = pd.DataFrame(y_pred)
df_y_test = pd.DataFrame(y_test).reset_index().drop(columns = 'index')
df_y_pred.columns = ['price_pred']
df_y_pred['upper'] = df_y_pred.price_pred + 30000
df_y_pred['lower'] = df_y_pred.price_pred - 30000
df_prediction = pd.concat([df_y_pred, df_y_test], axis = 1)
df_prediction['in_range'] = (df_prediction.price < df_prediction.upper) & (df_prediction.price > df_prediction.lower)

df_prediction

In [ ]:
df_prediction.in_range.sum() / len(df_prediction)

In [ ]:
df_prediction.columns

In [ ]:
plt.figure(figsize = (25,5))
df_prediction[['price_pred', 'price', 'upper', 'lower']].sample(5).sort_index().plot()

In [ ]:
diff = (y_pred - y_test.price)/y_test.price
sns.histplot(diff[(diff<2) & (diff >-2)])

In [ ]:
diff = (y_pred - y_test.price)/y_test.price
sns.histplot(diff)

## Another try

In [ ]:
df_y_pred = pd.DataFrame(y_pred)
df_y_test = pd.DataFrame(y_test).reset_index().drop(columns = 'index')
df_y_pred.columns = ['price_pred']
df_y_pred['upper'] = df_y_pred.price_pred*1.3
df_y_pred['lower'] = df_y_pred.price_pred*0.7
df_prediction = pd.concat([df_y_pred, df_y_test], axis = 1)
df_prediction['in_range'] = (df_prediction.price < df_prediction.upper) & (df_prediction.price > df_prediction.lower)

df_prediction.head()

In [ ]:
plt.figure(figsize = (25,5))
df_prediction[['price_pred', 'price', 'upper', 'lower']].sample(5).sort_index().plot()

In [ ]:
df_prediction.in_range.sum() / len(df_prediction)

In [ ]:
sns.scatterplot(y = df_prediction.price-df_prediction.price_pred, x = df_prediction.index, hue = df_prediction.in_range, alpha = 0.5)

In [ ]:
sns.scatterplot(df_prediction.price, df_prediction.price_pred, hue = df_prediction.in_range,alpha = 0.5)

## 📈 Linear Regression on data without outliers (df_without_outliers)

In [ ]:
from sklearn.linear_model import LinearRegression
model_lin = LinearRegression()

model_lin.fit(X_train_preproc, y_train)
y_pred = model_lin.predict(X_test_preproc)
mse_b = mean_squared_error(y_pred, y_test)
rmse_b = mse_b**(1/2)
print(f'Root Mean Squared Error: {rmse_b}')
print( 'Train score (R²):', model_lin.score(X_train_preproc, y_train))
print('Cross val score mean (R²) :', cross_val_score(model_lin, X_all, y_all, cv = 5).mean())

# No need anymore

## 🌳 RandomForest on full data of 93 (df_clean)

In [ ]:
#preprocess data
X_train_preproc_2, X_test_preproc_2, y_train_2, y_test_2, X_all_2, y_all_2, df_full_2 = preprocess_data(df_clean)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
model_rfr_2 = RandomForestRegressor()

model_rfr_2.fit(X_train_preproc_2, y_train_2)
y_pred_2 = model_rfr_2.predict(X_test_preproc_2)
mse_2a = mean_squared_error(y_test_2, y_pred_2)
rmse_2a = mse_2a**(1/2)
print(f'Root Mean Squared Error: {rmse_2a}')
print( 'Train score:', model_rfr_2.score(X_test_preproc_2, y_test_2))
print('Cross val score mean', cross_val_score(model_rfr_2, X_all_2, y_all_2, cv = 5).mean())

## 🪃 XGBoost on full data (df_clean)

In [ ]:
model_xgb_2 = XGBRegressor()

model_xgb_2.fit(X_train_preproc_2, y_train_2)
y_pred_2 = model_xgb_2.predict(X_test_preproc_2)
mse_2b = mean_squared_error(y_pred_2, y_test_2)
rmse_2b = mse_2b**(1/2)
print(f'Root Mean Squared Error: {rmse_2b}')
print( 'Train score:', model_xgb_2.score(X_train_preproc_2, y_train_2))
print('Cross val score mean (r2) :', cross_val_score(model_xgb_2, X_all_2, y_all_2, cv = 5, scoring = 'r2').mean())

## 📈 Linear Regression on full data (df_clean)

In [ ]:
from sklearn.linear_model import LinearRegression
model_lin = LinearRegression()

model_lin.fit(X_train_preproc_2, y_train_2)
y_pred_2 = model_lin.predict(X_test_preproc_2)
mse_2b = mean_squared_error(y_pred_2, y_test_2)
rmse_2b = mse_2b**(1/2)
print(f'Mean Squared Error: {rmse_2b}')
print( 'Train score:', model_lin.score(X_train_preproc_2, y_train_2))
print('Cross val score mean', cross_val_score(model_lin, X_all_2, y_all_2, cv = 5).mean())

## What model should we use ?

In [ ]:
model_xgb_2 = XGBRegressor()

model_xgb_2.fit(X_train_preproc_2, y_train_2)
y_pred_2 = model_xgb_2.predict(X_test_preproc_2)
mse_2b = mean_squared_error(y_pred_2, y_test_2)
rmse_2b = mse_2b**(1/2)
print(f'Root Mean Squared Error: {rmse_2b}')
print( 'Train score:', model_xgb_2.score(X_train_preproc_2, y_train_2))
print('Cross val score mean (r2) :', cross_val_score(model_xgb_2, X_all_2, y_all_2, cv = 5, scoring = 'r2').mean())

# download file

In [ ]:
df_without_outliers.to_pickle('../raw_data/dvf_93_cleaned.pkl')

# end of doc

In [ ]:
df_full_clean = pd.read_pickle('../raw_data/dvf_full_cleaned.pkl')

In [ ]:
df_full_clean.shape, df_full_clean.columns

In [ ]:
import sys
sys.path.append('/home/anouchka/code/qadnguyen/realdata')
from prop_value.ml_logic.preprocessor import preprocess_input
preprocess_input(df_full_clean)

In [ ]:
df93_clean = clean_data(df_93_raw)

In [ ]:
from xgboost import XGBRegressor
xgb_model_loaded = XGBRegressor()
xgb_model_loaded.load_model('../raw_data/xgb_model_2.bin')

In [ ]:
xgb_model_loaded

In [ ]:
pip install shap

In [ ]:
import shap
shap.initjs()

In [ ]:
X_test_preproc = pd.read_pickle('../raw_data/X_test_preproc')

In [ ]:
explainer = shap.Explainer(xgb_model_loaded)
shap_values = explainer(X_test_preproc)
shap.plots.beeswarm(shap_values)

In [ ]:
df_full_clean.shape